<a href="https://colab.research.google.com/github/enicaking/bluetab-uc3m-project-repo/blob/main/Bluetab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

<center>

<h1><b>Fraud Detection Project</b></h1>
<h3>Universidad Carlos III de Madrid · Bluetab</h3>

<p><em>Development of a predictive system using Machine Learning to identify fraudulent transactions and strengthen financial security.</em></p>

</center>

---

### Notebook Overview

This notebook is part of the *Bluetab–UC3M Fraud Detection Project*.  
Its main goal is to explore and merge the different datasets provided by the company (`customers_dirty.csv`, `transactions_dirty.csv`, `locations_dirty.csv`, and `creditcard.csv`) to build a clean and unified database for further analysis.  

Throughout this notebook, we:
- Load and inspect the raw data.  
- Perform initial cleaning and consistency checks.  
- Merge datasets into a single structured DataFrame.  
- EDA


---


In [ ]:
!pip install pvlib
!pip install catboost
!pip install chardet
!pip install xgboost
!pip install polars
!pip install lightgbm
!pip install tabulate
!pip install imbalanced-learn
!pip install scikit-learn
!pip install collections

ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections


In [ ]:
import sys
import time
import warnings
import chardet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import polars as pl
import os
from IPython.display import display

from datetime import datetime, timedelta
from itertools import combinations
from pytz import timezone

from tqdm import tqdm
from tqdm.auto import tqdm

from pvlib.location import Location
from scipy.stats import pearsonr


from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xgboost as xgb
import lightgbm as lgbm
from catboost import CatBoostRegressor, CatBoostClassifier

from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder
from imblearn.over_sampling import SMOTENC

np.random.seed(123)

# **Upload the datasets**

In [ ]:
# If working locally
# credit_df = pd.read_csv('content/creditcard.csv')
# transactions_df = pd.read_csv('content/transactions_dirty.csv')
# locations_df = pd.read_csv('content/locations_dirty.csv')
# customers_df = pd.read_csv('content/customers_dirty.csv')

In [ ]:
# If working on Colab
transactions_df = pd.read_csv('/content/transactions_dirty.csv')
locations_df = pd.read_csv('/content/locations_dirty.csv')
customers_df = pd.read_csv('/content/customers_dirty.csv')
flags_df = pd.read_csv('/content/flags.csv')
time_table_df = pd.read_csv('/content/time_table.csv')
devices_df = pd.read_csv('/content/devices.csv')

# **1. Raw Data Exploration**
This section focuses on the initial exploration of the four raw datasets provided by Bluetab: `customers_dirty.csv`, `transactions_dirty.csv`, `locations_dirty.csv`, and `creditcard.csv`.

The objective is to understand the structure, quality, and characteristics of each dataset, performing an EDA of each one. By analyzing them separately, we can identify inconsistencies, missing values, duplicates, and potential variables of interest for the fraud detection model.

## **Transactions**

In [ ]:
display(transactions_df.head())
transactions_df.shape

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V24,V25,V26,V27,V28,Amount,Class,transaction_id,customer_id,device_id
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0,822947e3-96bc-4124-8e77-75fc791a6308,1102,DEV_0004
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0,107b9ba3-0cd1-48ab-b548-ed6282de31af,1435,DEV_0039
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0,64a68036-03d7-434f-b4e9-6af563252772,1860,DEV_0035
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0,b3fcacd4-a564-479a-a9b9-cd04d10efed5,1270,DEV_0001
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0,2a51fbde-2f19-4ea5-bb7e-cefb24cabd36,1106,DEV_0024


(286815, 33)

In [ ]:
print(transactions_df['transaction_id'].duplicated().sum())

500


In [ ]:
transactions_df.drop_duplicates(inplace=True) # Complete equal rows are dropped
print(transactions_df['transaction_id'].duplicated().sum()) # There are 3 repeated ids in rows not completely equal

3


In [ ]:
duplicated_transactions = transactions_df[transactions_df['transaction_id'].duplicated(keep=False)] # Get all rows with duplicated transaction IDs
print("Duplicated transaction IDs in transactions_df:")
print(duplicated_transactions)

Duplicated transaction IDs in transactions_df:
              V1        V2        V3        V4        V5        V6        V7  \
13213  -2.039728  0.965692  2.166568 -0.005581 -0.086848 -0.098832 -0.216829   
31857  -1.185284  0.512991  2.057373  0.376168  0.234964  0.104018  0.140999   
128428 -1.236607  0.610138  1.576652  1.282104 -0.265792 -0.374496 -0.268472   
286366 -1.236607  0.610138  1.576652  1.282104 -0.265792 -0.374496 -0.268472   
286537 -2.039728  0.965692  2.166568 -0.005581 -0.086848 -0.098832 -0.216829   
286783 -1.185284  0.512991  2.057373  0.376168  0.234964  0.104018  0.140999   

              V8        V9       V10  ...       V24       V25       V26  \
13213   0.198205  1.605322  0.364846  ...  0.157698 -0.031629  0.211652   
31857   0.329286 -0.358042 -0.077797  ... -0.025780 -0.152223 -0.523562   
128428  0.471094 -0.561624 -0.352570  ...  0.454560 -0.392421 -0.232860   
286366  0.471094 -0.561624 -0.352570  ...  0.454560 -0.392421 -0.232860   
286537  0.198205 

The amount is the same, the customer and device id are different and it is impossible to have two equal transaction ids, so we compared with other database.

In [ ]:
duplicated_df = transactions_df[transactions_df['transaction_id'].duplicated(keep=False)].copy()
aux = pd.merge(duplicated_df, customers_df, on='customer_id', how='inner')
display(aux)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,transaction_id,customer_id,device_id,name,age,email,phone,country,credit_score,join_date
0,-2.039728,0.965692,2.166568,-0.005581,-0.086848,-0.098832,-0.216829,0.198205,1.605322,0.364846,...,a995c6a8-ef9d-4c4f-928d-7149a5549fc8,1163,DEV_0100,Ariel Sandoval,70,NaN,689.731.8692x8972,Serbia,787,2021-09-17
1,-1.236607,0.610138,1.576652,1.282104,-0.265792,-0.374496,-0.268472,0.471094,-0.561624,-0.352570,...,70a09c87-2693-4455-9373-01c07f4cbc65,1955,DEV_0089,Anthony Rodriguez,38,xmalone@example.org,611-818-2422x412,Greenland,702,2023-08-27
2,-1.185284,0.512991,2.057373,0.376168,0.234964,0.104018,0.140999,0.329286,-0.358042,-0.077797,...,7dd260b9-5836-4d26-9163-ceff19cee458,1617,DEV_0038,Virginia Casey,33,carpenterjorge@example.net,001-821-229-9642,Luxembourg,493,2022-06-20


In [ ]:
# Remove the rows which are incorrect (the customer_id does not match in the transaction database with the customer database)
transactions_df = transactions_df[~(
    (transactions_df['transaction_id'] == "a995c6a8-ef9d-4c4f-928d-7149a5549fc8") &
    (transactions_df['customer_id'] == 99180)
)]

transactions_df = transactions_df[~(
    (transactions_df['transaction_id'] == "70a09c87-2693-4455-9373-01c07f4cbc65") &
    (transactions_df['customer_id'] == 99172)
)]

transactions_df = transactions_df[~(
    (transactions_df['transaction_id'] == "7dd260b9-5836-4d26-9163-ceff19cee458") &
    (transactions_df['customer_id'] == 99209)
)]

In [ ]:
print(transactions_df.shape)

(286315, 33)


## **Locations**

In [ ]:
display(locations_df.head())
locations_df.shape

,transaction_id,ip_address,country,city,zip_code,merchant
0,822947e3-96bc-4124-8e77-75fc791a6308,83.63.232.13,Australia,Perth,NaN,Davis-Parker
1,107b9ba3-0cd1-48ab-b548-ed6282de31af,89.213.161.51,Australia,Perth,57376.0,Garcia-Mitchell
2,64a68036-03d7-434f-b4e9-6af563252772,140.231.3.186,France,Lyon,19277.0,Mccann-Snow
3,b3fcacd4-a564-479a-a9b9-cd04d10efed5,40.46.209.108,Germany,Berlin,93260.0,Jones Inc
4,2a51fbde-2f19-4ea5-bb7e-cefb24cabd36,174.155.184.108,Australia,Melbourne,97401.0,Rodriguez Inc


(286315, 6)

In [ ]:
print(locations_df['transaction_id'].duplicated().sum())

0


## **Customers**

In [ ]:
display(customers_df.head())
customers_df.shape

,customer_id,name,age,email,phone,country,credit_score,join_date
0,1102,Allison Hill,56,qjacobson@example.org,6502166799,Gambia,787,2024-03-18
1,1435,Noah Rhodes,69,ylopez@example.com,767.389.9730x8069,Georgia,582,2021-09-04
2,1860,Angie Henderson,46,seanbaker@example.com,567-917-9576x024,Mayotte,765,2023-11-19
3,1270,Daniel Wagner,32,jeremy49@example.org,(873)783-9597x246,Papua New Guinea,612,2021-02-23
4,1106,Cristian Santos,60,karencontreras@example.org,329.834.1722x3297,Serbia,580,2021-04-17


(1000, 8)

In [ ]:
print(customers_df['customer_id'].duplicated().sum())

0


## **Flags**

In [ ]:
display(flags_df.head())
flags_df.shape

,transaction_id,is_foreign_tx,is_night_tx,is_high_amount
0,822947e3-96bc-4124-8e77-75fc791a6308,1,1,0
1,107b9ba3-0cd1-48ab-b548-ed6282de31af,1,1,0
2,64a68036-03d7-434f-b4e9-6af563252772,1,1,0
3,b3fcacd4-a564-479a-a9b9-cd04d10efed5,1,1,0
4,2a51fbde-2f19-4ea5-bb7e-cefb24cabd36,1,1,0


(286315, 4)

In [ ]:
print(flags_df['transaction_id'].duplicated().sum())

0


## **Time-table**

In [24]:
display(time_table_df.head())
time_table_df.shape

,transaction_id,timestamp,hour,day_of_week,is_weekend,month,time_of_day
0,822947e3-96bc-4124-8e77-75fc791a6308,2023-01-01 00:00:00,0,6,1,1,Night
1,107b9ba3-0cd1-48ab-b548-ed6282de31af,2023-01-01 00:00:00,0,6,1,1,Night
2,64a68036-03d7-434f-b4e9-6af563252772,2023-01-01 00:00:01,0,6,1,1,Night
3,b3fcacd4-a564-479a-a9b9-cd04d10efed5,2023-01-01 00:00:01,0,6,1,1,Night
4,2a51fbde-2f19-4ea5-bb7e-cefb24cabd36,2023-01-01 00:00:02,0,6,1,1,Night


(286315, 7)

In [ ]:
print(time_table_df['transaction_id'].duplicated().sum())

0


## **Devices**

In [ ]:
display(devices_df.head())
devices_df.shape

,device_id,device_type,os,browser,is_mobile
0,DEV_0001,Desktop,macOS,Chrome,0
1,DEV_0002,POS,Android,Firefox,0
2,DEV_0003,ATM,Linux,Chrome,0
3,DEV_0004,Desktop,Windows,Safari,0
4,DEV_0005,POS,Android,Safari,0


(100, 5)

In [ ]:
print(devices_df['device_id'].duplicated().sum())

0


# **2. Data Merging and Integration**
This section combines the different raw datasets into a single unified structure, linking transactions, customers, locations (*TODO: and credit card features.*)
The goal is to build a consistent and complete dataset that can be used for global exploratory analysis and model development.

In [28]:
merged_df = pd.merge(transactions_df, locations_df, on='transaction_id', how='inner')
print(merged_df.shape)

(286315, 38)


In [29]:
merge2_df = pd.merge(merged_df, flags_df, on='transaction_id', how='inner')
display(merge2_df.head())
print(merge2_df.shape)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,customer_id,device_id,ip_address,country,city,zip_code,merchant,is_foreign_tx,is_night_tx,is_high_amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,1102,DEV_0004,83.63.232.13,Australia,Perth,NaN,Davis-Parker,1,1,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,1435,DEV_0039,89.213.161.51,Australia,Perth,57376.0,Garcia-Mitchell,1,1,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,1860,DEV_0035,140.231.3.186,France,Lyon,19277.0,Mccann-Snow,1,1,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,1270,DEV_0001,40.46.209.108,Germany,Berlin,93260.0,Jones Inc,1,1,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,1106,DEV_0024,174.155.184.108,Australia,Melbourne,97401.0,Rodriguez Inc,1,1,0


(286315, 41)


In [30]:
merge3_df = pd.merge(merge2_df, time_table_df, on='transaction_id', how='inner')
display(merge3_df.head())
print(merge3_df.shape)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,merchant,is_foreign_tx,is_night_tx,is_high_amount,timestamp,hour,day_of_week,is_weekend,month,time_of_day
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,Davis-Parker,1,1,0,2023-01-01 00:00:00,0,6,1,1,Night
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,Garcia-Mitchell,1,1,0,2023-01-01 00:00:00,0,6,1,1,Night
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,Mccann-Snow,1,1,0,2023-01-01 00:00:01,0,6,1,1,Night
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,Jones Inc,1,1,0,2023-01-01 00:00:01,0,6,1,1,Night
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,Rodriguez Inc,1,1,0,2023-01-01 00:00:02,0,6,1,1,Night


(286315, 47)


In [ ]:
merge4_df = pd.merge(merge3_df, devices_df, on='device_id', how='inner')
display(merge4_df.head())
print(merge4_df.shape)

In [26]:
df = pd.merge(merge4_df, customers_df, on='customer_id', how='inner')
display(df.head())
print(df.shape)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,city,zip_code,merchant,name,age,email,phone,country_y,credit_score,join_date
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,Perth,NaN,Davis-Parker,Allison Hill,56,qjacobson@example.org,6502166799,Gambia,787,2024-03-18
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,Perth,57376.0,Garcia-Mitchell,Noah Rhodes,69,ylopez@example.com,767.389.9730x8069,Georgia,582,2021-09-04
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,Lyon,19277.0,Mccann-Snow,Angie Henderson,46,seanbaker@example.com,567-917-9576x024,Mayotte,765,2023-11-19
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,Berlin,93260.0,Jones Inc,Daniel Wagner,32,jeremy49@example.org,(873)783-9597x246,Papua New Guinea,612,2021-02-23
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,Melbourne,97401.0,Rodriguez Inc,Cristian Santos,60,karencontreras@example.org,329.834.1722x3297,Serbia,580,2021-04-17


(286068, 45)
